# Etude du problème d'optimisation

Question 1.1  
La définition de l'énergie auto-consommée (3) est interprétable de la manière suivante :  

$E^{PV}$ représente l'énergie produite par les panneaux photovoltaïques et ($w_{i}$ + $P_{i}\Delta$t) la somme des consommations non-pilotables de la maison et de l'énergie dépensée pour chauffer l'eau du ballon. Donc pour chaque i (chaque intervalle de temps) nous pouvons prendre le minimum des deux variables car nous avons trois cas :  
- Si $w_{i}$ + $P_{i}\Delta$ t > $E^{PV}_{i}$
Alors la totalité de l'énergie photovoltaïque a été consommée, on la comptabilise entièrement pour l'intervalle "i" concerné.

- Si $w_{i}$ + $P_{i}\Delta$ t < $E^{PV}_{i}$
Alors nous n'avons pas consommé la totalité de l'énergie photovoltaïque produite et le surplus de production a été revendu. Donc nous pouvons comptabiliser le terme minimal ($w_{i}$ + $P_{i}\Delta$t) qui est égal à la consommation d'énergie photovoltaïque.

- Si $w_{i}$ + $P_{i}\Delta$ t = $E^{PV}_{i}$

Question 1.2

Supposons que $Ê_i < P_i \Delta t$ par symétrie (h est symétrique en $Ê_i$ et $P_i \Delta t$). 

On a donc $h(Ê_i, P_i \Delta t) = \frac{Ê_i + P_i \Delta te^{- \alpha (P_i\Delta t -Ê_i)}}{1+e^{- \alpha (P_i\Delta t-Ê_i)}}$.

Or $ \alpha (P_i \Delta t -Ê_i) >>1$ donc $e^{- \alpha (P_i\Delta t -Ê_i)} <<1$.

Par développement limité à l'ordre 0 exacte de la focntion $\frac{1}{1+x}$ et $1+x$, on a donc: $h(Ê_i, P_i \Delta t) = (Ê_i +0)*(1+0) = Ê_i$. Par symétrie, on a donc qu'à l'ordre 0 :
$$ h(Ê_i, P_i \Delta t) = min(Ê_i, P_i \Delta t)$$

L'avantage de la fonction h est qu'elle est $C^1$ et explicite, donc beaucoup plus facile à manipuler que la fonction minimum.

Question 1.3

Question 1.4